In [2]:
!nvidia-smi

Thu May 25 11:59:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!sudo apt-get install subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 libutf8proc2 subversion
0 upgraded, 6 newly installed, 0 to remove and 28 not upgraded.
Need to get 2,355 kB of archives.
After this operation, 10.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libapr1 amd64 1.6.5-1ubuntu1 [91.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libaprutil1 amd64 1.6.1-4ubuntu2.1 [84.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libserf-1-1 amd64 1.3.9-8build1 [45.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libutf8proc2 amd64 2.5.0-1 [50.0 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-

In [3]:
!svn checkout https://github.com/omasaht/headpose-fsanet-pytorch/trunk/checkpoints

A    checkpoints/fsa1x1-08082020.chkpt
A    checkpoints/fsavar-09082020.chkpt
Checked out revision 15.


In [4]:
!wget https://github.com/omasaht/headpose-fsanet-pytorch/raw/master/src/dataset.py

--2023-05-25 12:00:03--  https://github.com/omasaht/headpose-fsanet-pytorch/raw/master/src/dataset.py
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/omasaht/headpose-fsanet-pytorch/master/src/dataset.py [following]
--2023-05-25 12:00:03--  https://raw.githubusercontent.com/omasaht/headpose-fsanet-pytorch/master/src/dataset.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1885 (1.8K) [text/plain]
Saving to: ‘dataset.py’

dataset.py          100%[===================>]   1.84K  --.-KB/s    in 0s      

2023-05-25 12:00:04 (34.3 MB/s) - ‘dataset.py’ saved [1885/1885]



In [5]:
!wget https://github.com/omasaht/headpose-fsanet-pytorch/raw/master/src/model.py

--2023-05-25 12:00:08--  https://github.com/omasaht/headpose-fsanet-pytorch/raw/master/src/model.py
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/omasaht/headpose-fsanet-pytorch/master/src/model.py [following]
--2023-05-25 12:00:08--  https://raw.githubusercontent.com/omasaht/headpose-fsanet-pytorch/master/src/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16800 (16K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]  16.41K  --.-KB/s    in 0s      

2023-05-25 12:00:08 (116 MB/s) - ‘model.py’ saved [16800/16800]



In [6]:
!cd checkpoints ; ls

fsa1x1-08082020.chkpt  fsavar-09082020.chkpt


In [7]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 86.4 MB/s eta 0:00:00


In [8]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.8 MB/s eta 0:00:00


# Export to ONNX

In this notebook, we export our pytorch model to ONNX so that it can later be used for inference.

**1. Import Required Libraries:-** 

In [1]:
from torchvision import transforms
import torch
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import time
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
#Local Imports
from dataset import HeadposeDataset
from model import FSANet
import onnx
import onnxruntime


**2. Define Model and Load from Saved Checkpoint:-**

In [3]:
device = torch.device("cuda")
model = FSANet(var=False).to(device)
#Load Model Checkpoint
chkpt_dic = torch.load('checkpoints/fsa1x1-08082020.chkpt')

model.load_state_dict(chkpt_dic['best_states']['model'])
#set model to inference-ready
model.eval()

FSANet(
  (msms): MultiStreamMultiStage(
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (s0_conv0): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (pointwise): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
    (s0_conv1_0): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (pointwise): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
    (s0_conv1_1): SepConvBlock(
      (conv): SepConv2d(
        (depthwise): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (pointwise): Conv2d(32, 32, kernel_si

In [4]:
!mkdir pretrained

mkdir: cannot create directory ‘pretrained’: File exists


**3. Export model to ONNX:-**

In [5]:
#Export to ONNX
x = torch.randn(1,3,64,64).to(device)
model_out = model(x)
save_path = "pretrained/fsanet-1x1-iter-688590.onnx"

torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  save_path,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:1248: UserWarning: This model contains a squeeze operation on dimension 3. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/onnx/symbolic_opset9.py:1248: UserWarning: This model contains a squeeze operation on dimension 2. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [6]:
!cd pretrained; ls

fsanet-1x1-iter-688590.onnx  fsanet-var-iter-688590.onnx


REPEAT THIS STEP WITH "fsanet-var-iter-688590.onnx" TOO. PLEASE 

1. AMEND THE HARCODED CHECKPOINT FILE PATH
2. AMEND CONSTRUCTOR ARGUMENT VAR=True
3. AMEND THE HARDCODED EXPORT PATH
4. RESTART THE GPU INSTANCE RUNTIME